En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

In [2]:
file_path = "farmers-protest-tweets-2021-2-4.json"

**Autor:** Leonardo Burbano
<br/>
**Empresa:** TW
<br/>
**Email:** leonardo.burbano@[dominioTW]
<br/>
**Mes/Año:** 05/2023

#### 1. Suposiciones

- La descarga/carga del archivo de input no se realizará de forma recurrente. Se realizará una sola vez para efectos de solucionar el presente desafío.
- La ejecución de la descarga/descompresión/carga del archivo se realizará bajo demanda (y manualmente de ser el caso). Se considerará como variable el nombre del archivo, y como ubicación fija un bucket (repositorio) en Cloud Storage (GCP - Google Cloud Platform).
- La prioridad es obtener la respuesta a las preguntas planteadas. No surgirán otras preguntas en el corto plazo.
- Al final de este documento se detallan las recomendaciones para garantizar la escalabilidad del proceso y también la calendarización del mismo de ser necesario.
- La respuesta proviene de una consulta a la API de Twitter que está almacenada en un archivo comprimido en Google Drive.
- El desarrollo no incluye versionamiento de la infraestructura creada o utilización de infraestructura como código.
- No se incluyen configuraciones de permisos y otras tareas de disposición de la infraestructura en GCP.

#### 2. Configuración del ambiente
- Se utilizó Git y Gitflow para Mac, para facilitar el versionamiento y simulación de trabajo colaborativo.
- Se realizó una exploración inicial en la que se fue
- Se utilizará la nube de GCP, se requieré únicamente un archivo de cuenta de servicio con los permisos adecuados para ejecutar el proyecto.


#### 2. Estrategia y features a implementar

##### 2.1. **feature/initial_exploration:** 
Se comienza tratando de llegar a la respuesta a las preguntas lo más rápido posible utilizando un Notebook de exploración y las configuraciones manuales en GCP (ejemplo: Cargar un archivo a Cloud Storage, cargar el archivo a BigQuery). Este tipo de pruebas dieron mayor visibilidad del esquema del archivo y los posibles desafíos a enfrentar. El detalle se encuentra en ../_initial_exploration/00_test.ipynb . A partir de esa exploración se realizó esta estrategia y varias de las decisiones tomadas a continuación.

##### 2.2 **feature/environment_setup:** 
Se realiza la configuración del ambiente en GCP creando una "Service Account" en la sección de "IAM" con los permisos necesarios para cargar datos en BigQuery, Cloud Storage y otros servicios utilizados en esta solución. Adicionalmente, para el ambiente local, se considera diferentes configuraciones utilizando Git, Github, Gitflow y Visual Studio Code. Como requisitos generales, se utiliza: Python 3.9.6 en una MacBook Pro (M1 Pro, 16 Gb RAM, Sonoma).

##### 2.3. **feature/bigquery_connector:** 
Considerando la carga manual realizada y el desarrollo realizado en el punto 2.1. se inicia desarrollando un conector a BigQuery que permita ejecutar las consultas ya desarrolladas en la exploración inicial.


#### 3. Descarga del archivo, descompresión y cargar en Cloud Storage


#### 4. Testing básico de las funciones

In [3]:
from q1_memory import q1_memory
from q1_time import q1_time
from q2_memory import q2_memory
from q2_time import q2_time
from q3_memory import q3_memory
from q3_time import q3_time

print(q1_memory(file_path))
print(q1_time(file_path))
print(q2_memory(file_path))
print(q2_time(file_path))
print(q3_memory(file_path))
print(q3_time(file_path))

Hola
None
None
None
None
None
None
